In [1]:
# coding: utf-8
import os

import pandas as pd
import numpy as np
import scipy
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

from mydatools.plot import plot_multiclass_feature_dist
from mydatools.features_generate import mean_encoding

% matplotlib inline

## Config

In [2]:
trn_path = './data/input/application_train.csv'
tst_path = './data/input/application_test.csv'
id_col = 'SK_ID_CURR'
label_col = 'TARGET'

submission_path = './data/output/submission/submission.csv'
output_id_col = id_col
output_label_col = label_col

## Load Data

In [3]:
trn_df = pd.read_csv(trn_path)
trn_df['ds_type'] = 'trn'
trn_df[label_col] = trn_df[label_col].astype('int')

tst_df = pd.read_csv(tst_path)
tst_df['ds_type'] = 'tst'

full_df = pd.concat([trn_df, tst_df])

del(trn_df)
del(tst_df)

full_df.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,ds_type
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0149,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341,trn
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0714,Block,MONDAY,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040,trn
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn


In [4]:
# dataset type
is_trn = full_df['ds_type'] == 'trn'
is_tst = full_df['ds_type'] == 'tst'

In [5]:
full_df.shape

(356255, 123)

**label非常不平衡，大约10: 1**

In [6]:
full_df[label_col].value_counts()

0.0    282686
1.0     24825
Name: TARGET, dtype: int64

## Features

In [7]:
feature_columns = []

def add_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns.extend([f for f in features if f not in feature_columns])

### application_train/test

**numerical_features**

In [8]:
numerical_features = full_df.dtypes[full_df.dtypes != 'object'].index.tolist()
numerical_features = [c for c in numerical_features if c not in [id_col, label_col, 'ds_type']]
add_features(numerical_features)

In [9]:
full_df[numerical_features].describe()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,TOTALAREA_MODE,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE
count,356219.000000,3.562550e+05,3.559770e+05,3.562550e+05,308687.000000,308687.000000,308687.000000,308687.000000,308687.000000,308687.000000,...,356255.000000,356255.000000,356255.000000,185200.000000,183392.000000,183392.000000,183392.000000,119949.000000,119949.000000,119949.000000
mean,27425.560657,5.877674e+05,5.280200e+05,1.701161e+05,0.006281,0.005808,0.231697,0.304399,0.029995,1.911564,...,0.229661,0.015649,0.051371,0.103193,0.977889,0.977903,0.977239,0.752283,0.755548,0.759452
std,14732.808190,3.986237e+05,3.660650e+05,2.235068e+05,0.104250,0.079736,0.855949,0.786915,0.191374,1.865338,...,0.420616,0.124113,0.220753,0.108041,0.057929,0.058562,0.063165,0.113267,0.112057,0.110112
min,1615.500000,4.500000e+04,4.050000e+04,2.565000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16731.000000,2.700000e+05,2.340000e+05,1.125000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.041500,0.976700,0.976700,0.976700,0.687200,0.691400,0.699400
50%,25078.500000,5.002110e+05,4.500000e+05,1.530000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.069000,0.981600,0.981600,0.981600,0.755200,0.758500,0.764800
75%,34960.500000,7.975575e+05,6.750000e+05,2.025000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.128700,0.986600,0.986600,0.986600,0.823200,0.825600,0.823600
max,258025.500000,4.050000e+06,4.050000e+06,1.170000e+08,9.000000,4.000000,27.000000,261.000000,8.000000,25.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


**categorical_features**

In [10]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
full_df[categorical_features].nunique()

CODE_GENDER                    3
EMERGENCYSTATE_MODE            2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
NAME_CONTRACT_TYPE             2
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_INCOME_TYPE               8
NAME_TYPE_SUITE                7
OCCUPATION_TYPE               18
ORGANIZATION_TYPE             58
WALLSMATERIAL_MODE             7
WEEKDAY_APPR_PROCESS_START     7
ds_type                        2
dtype: int64

In [11]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
categorical_features = categorical_features[:-1] # 为了去掉ds_type
# 去掉分类数太多
# categorical_features = full_df[categorical_features].columns[full_df[categorical_features].nunique() <= 10]

# get dummies
temp_df = full_df[categorical_features].copy()
temp_df = pd.get_dummies(temp_df, prefix_sep=':')
for c in temp_df.columns:
    full_df[c] = temp_df[c]

# add features
add_features(temp_df.columns.tolist())

增加分类变量的mean-encoding，因为训练集和测试集完全随机分割，随意直接使用最普通的mean-encoding应该也不太会出现过拟合

In [12]:
for c in categorical_features:
    full_df, new_feat = mean_encoding(full_df, c, label_col, is_tst, return_feature_name=True)
    add_features(new_feat)

### add extra data

In [13]:
def summary_extra_data(df, suffix):
    # 分类特征get_dummies
    df = pd.get_dummies(df, columns=df.columns[df.dtypes == 'object'].tolist())
    df = df.fillna(0)

    # 求平均
    df = pd.concat([
        df.groupby('SK_ID_CURR').mean(),
        df.groupby('SK_ID_CURR').size().rename('cnt'),
    ], axis=1)

    # 给特征加上前缀
    df.columns = [suffix+f for f in df.columns.tolist()]

    return df

In [14]:
# bureau = pd.read_csv('./data/input/bureau.csv')
# test_df = bureau.iloc[:100, :5]
# test_df.head()

# summary_extra_data(test_df, 'test')

**bureau.csv**

In [15]:
bureau = pd.read_csv('./data/input/bureau.csv')
bureau = bureau.drop('SK_ID_BUREAU', axis=1)

bureau = summary_extra_data(bureau, 'bureau_')

full_df = full_df.join(bureau, on='SK_ID_CURR')
add_features(bureau.columns.tolist())

**previous_application.csv**

In [16]:
prevapp = pd.read_csv('./data/input/previous_application.csv')
prevapp = prevapp.drop('SK_ID_PREV', axis=1)

prevapp = summary_extra_data(prevapp, 'prevapp_')

full_df = full_df.join(prevapp, on='SK_ID_CURR')
add_features(prevapp.columns.tolist())

**POS_CASH_balance.csv**

In [17]:
pcblc = pd.read_csv('./data/input/POS_CASH_balance.csv')
pcblc = pcblc.drop(['SK_ID_PREV', 'MONTHS_BALANCE'], axis=1)

pcblc = summary_extra_data(pcblc, 'pcblc_')

full_df = full_df.join(pcblc, on='SK_ID_CURR')
add_features(pcblc.columns.tolist())

**credit_card_balance.csv**

In [18]:
ccblc = pd.read_csv('./data/input/credit_card_balance.csv')
ccblc = ccblc.drop(['SK_ID_PREV', 'MONTHS_BALANCE'], axis=1)

ccblc = summary_extra_data(ccblc, 'ccblc_')

full_df = full_df.join(ccblc, on='SK_ID_CURR')
add_features(ccblc.columns.tolist())

**installments_payments.csv**

In [19]:
installpay = pd.read_csv('./data/input/installments_payments.csv')
installpay = installpay.drop(['SK_ID_PREV'], axis=1)

installpay = summary_extra_data(installpay, 'installpay_')

full_df = full_df.join(installpay, on='SK_ID_CURR')
add_features(installpay.columns.tolist())

In [20]:
len(feature_columns)

507

## LightGBM

In [21]:
trn_df = full_df[is_trn]
tst_df = full_df[is_tst]

X = trn_df[feature_columns]
y = trn_df[label_col]
X_trn, X_val, y_trn, y_val = train_test_split(X, y, test_size=0.20, random_state=20180521)

X_tst = tst_df[feature_columns]

d_trn = lgb.Dataset(X_trn.values, y_trn)
d_val = lgb.Dataset(X_val.values, y_val)

In [22]:
params = {
    'application': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 4,
    'num_leaves': 12,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'is_unbalance': True,
    'seed': 0,
    'reg_alpha': 0.1,
}
bst = lgb.train(params, d_trn, 500, valid_sets=[d_trn, d_val], early_stopping_rounds=20)

[1]	training's auc: 0.69531	valid_1's auc: 0.69813
Training until validation scores don't improve for 20 rounds.
[2]	training's auc: 0.704679	valid_1's auc: 0.70877
[3]	training's auc: 0.714386	valid_1's auc: 0.718351
[4]	training's auc: 0.717777	valid_1's auc: 0.72229
[5]	training's auc: 0.719933	valid_1's auc: 0.724448
[6]	training's auc: 0.720261	valid_1's auc: 0.724702
[7]	training's auc: 0.720914	valid_1's auc: 0.725598
[8]	training's auc: 0.720845	valid_1's auc: 0.725313
[9]	training's auc: 0.723225	valid_1's auc: 0.727539
[10]	training's auc: 0.724181	valid_1's auc: 0.728151
[11]	training's auc: 0.724522	valid_1's auc: 0.729004
[12]	training's auc: 0.726765	valid_1's auc: 0.730966
[13]	training's auc: 0.728629	valid_1's auc: 0.733054
[14]	training's auc: 0.729136	valid_1's auc: 0.733683
[15]	training's auc: 0.730057	valid_1's auc: 0.73451
[16]	training's auc: 0.730439	valid_1's auc: 0.73477
[17]	training's auc: 0.730364	valid_1's auc: 0.734864
[18]	training's auc: 0.730578	valid

[152]	training's auc: 0.780241	valid_1's auc: 0.774006
[153]	training's auc: 0.780458	valid_1's auc: 0.774155
[154]	training's auc: 0.780623	valid_1's auc: 0.774213
[155]	training's auc: 0.780674	valid_1's auc: 0.774228
[156]	training's auc: 0.78085	valid_1's auc: 0.774336
[157]	training's auc: 0.781005	valid_1's auc: 0.774376
[158]	training's auc: 0.78115	valid_1's auc: 0.774416
[159]	training's auc: 0.781284	valid_1's auc: 0.77452
[160]	training's auc: 0.781427	valid_1's auc: 0.774496
[161]	training's auc: 0.781531	valid_1's auc: 0.774504
[162]	training's auc: 0.781733	valid_1's auc: 0.77468
[163]	training's auc: 0.781898	valid_1's auc: 0.774767
[164]	training's auc: 0.782025	valid_1's auc: 0.774865
[165]	training's auc: 0.782155	valid_1's auc: 0.774949
[166]	training's auc: 0.782242	valid_1's auc: 0.774951
[167]	training's auc: 0.782363	valid_1's auc: 0.774956
[168]	training's auc: 0.782497	valid_1's auc: 0.775016
[169]	training's auc: 0.782599	valid_1's auc: 0.775063
[170]	training

[302]	training's auc: 0.797133	valid_1's auc: 0.78088
[303]	training's auc: 0.797207	valid_1's auc: 0.780917
[304]	training's auc: 0.797326	valid_1's auc: 0.780939
[305]	training's auc: 0.797444	valid_1's auc: 0.780942
[306]	training's auc: 0.797515	valid_1's auc: 0.780962
[307]	training's auc: 0.797587	valid_1's auc: 0.780977
[308]	training's auc: 0.7977	valid_1's auc: 0.781074
[309]	training's auc: 0.797792	valid_1's auc: 0.781087
[310]	training's auc: 0.797861	valid_1's auc: 0.781059
[311]	training's auc: 0.797968	valid_1's auc: 0.781081
[312]	training's auc: 0.798031	valid_1's auc: 0.781124
[313]	training's auc: 0.79816	valid_1's auc: 0.781158
[314]	training's auc: 0.798229	valid_1's auc: 0.781227
[315]	training's auc: 0.798294	valid_1's auc: 0.781247
[316]	training's auc: 0.798364	valid_1's auc: 0.781212
[317]	training's auc: 0.798445	valid_1's auc: 0.78121
[318]	training's auc: 0.798517	valid_1's auc: 0.781235
[319]	training's auc: 0.798561	valid_1's auc: 0.781249
[320]	training'

[452]	training's auc: 0.808745	valid_1's auc: 0.783143
[453]	training's auc: 0.808826	valid_1's auc: 0.783175
[454]	training's auc: 0.808896	valid_1's auc: 0.783146
[455]	training's auc: 0.808954	valid_1's auc: 0.783168
[456]	training's auc: 0.809013	valid_1's auc: 0.783149
[457]	training's auc: 0.809053	valid_1's auc: 0.783133
[458]	training's auc: 0.809082	valid_1's auc: 0.783153
[459]	training's auc: 0.809102	valid_1's auc: 0.783163
[460]	training's auc: 0.809156	valid_1's auc: 0.783184
[461]	training's auc: 0.809208	valid_1's auc: 0.783238
[462]	training's auc: 0.809276	valid_1's auc: 0.783263
[463]	training's auc: 0.80938	valid_1's auc: 0.783306
[464]	training's auc: 0.809443	valid_1's auc: 0.783323
[465]	training's auc: 0.809509	valid_1's auc: 0.783295
[466]	training's auc: 0.809585	valid_1's auc: 0.783253
[467]	training's auc: 0.809667	valid_1's auc: 0.783277
[468]	training's auc: 0.809772	valid_1's auc: 0.783319
[469]	training's auc: 0.809806	valid_1's auc: 0.783313
[470]	train

In [24]:
imp_df = pd.DataFrame([bst.feature_importance()], columns=feature_columns, index=['importance']).T.sort_values(by='importance', ascending=False)
imp_df

,importance
EXT_SOURCE_2,210
EXT_SOURCE_3,191
EXT_SOURCE_1,167
DAYS_BIRTH,129
AMT_CREDIT,116
pcblc_CNT_INSTALMENT_FUTURE,111
AMT_ANNUITY,107
AMT_GOODS_PRICE,89
DAYS_EMPLOYED,89
bureau_AMT_CREDIT_SUM_DEBT,85


## predict

In [25]:
res_df = pd.DataFrame(bst.predict(X_tst), columns=[output_label_col])
res_df[output_id_col] = tst_df[output_id_col].values
res_df[[output_id_col, output_label_col]].to_csv(submission_path, index=False)